In [1]:
'''
Mulliken population analysis with IAO orbitals
'''

import numpy
from functools import reduce
from sys import stdout
from pyscf import gto, scf, lo
from pyscf.tools.dump_mat import dump_rec
from pyscf.tools.dump_mat import dump_mo

mol = gto.M(atom="""
  C1    0.0000000   -0.0000000   -0.6657255
  H1    0.0000000    0.9361253   -1.2138526
  H1   -0.0000000   -0.9361253   -1.2138526
  C2   -0.0000000    0.0000000    0.6657255
  H2    0.0000000    0.9361253    1.2138526
  H2   -0.0000000   -0.9361253    1.2138526
""", basis='cc-pvdz')

mol.build()

nao_h2o = mol.nao // 2

h2o_1_idx = gto.search_ao_label(mol, ["C1", "H1"])
h2o_2_idx = gto.search_ao_label(mol, ["C2", "H2"])

print(h2o_1_idx)
print(h2o_2_idx)

mf = scf.RHF(mol).run()
c_ao_lo = lo.orth_ao(mf, 'nao')

assert numpy.linalg.norm(reduce(numpy.dot, [c_ao_lo.T, mf.get_ovlp(), c_ao_lo]) - numpy.eye(c_ao_lo.shape[0])) < 1e-8

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
converged SCF energy = -78.0396820024877


In [2]:
# check the localality
from scipy.linalg import sqrtm
ww = numpy.einsum("mn,np->mp", sqrtm(mf.get_ovlp()), c_ao_lo)
w2 = ww**2

for i in range(nao_h2o):
    assert abs(1.0 - sum(w2[h2o_1_idx, i])) < 2e-1
    assert abs(sum(w2[h2o_2_idx, i])) < 2e-1
    assert abs(1.0 - sum(w2[h2o_2_idx, i + nao_h2o])) < 2e-1
    assert abs(sum(w2[h2o_1_idx, i + nao_h2o])) < 2e-1

In [3]:
# transform mo_occ to IAO representation. Note the AO dimension is reduced
ovlp_ao = mf.get_ovlp()
dm_ao = mf.make_rdm1()
dm_lo = reduce(numpy.dot, [c_ao_lo.T, ovlp_ao, dm_ao, ovlp_ao, c_ao_lo])
assert numpy.linalg.norm(reduce(numpy.dot, [dm_lo/2, dm_lo/2]) - dm_lo/2) < 1e-8

idx_a = h2o_1_idx
idx_b = h2o_2_idx

dm_aa = dm_lo[numpy.ix_(idx_a, idx_a)]
dm_bb = dm_lo[numpy.ix_(idx_b, idx_b)]
dm_ab = dm_lo[numpy.ix_(idx_a, idx_b)]

In [4]:
from scipy import linalg

uab, sab, vhab = linalg.svd(dm_ab)
numpy.linalg.norm(
    dm_ab - reduce(numpy.dot, [uab, numpy.diag(sab), vhab])
)

2.0554372585978094e-15

In [5]:
from pyscf import tools

ua, sa, vha = linalg.svd(dm_aa)
numpy.linalg.norm(
    dm_aa - reduce(numpy.dot, [ua, numpy.diag(sa), vha])
)

tol = 1e-2
cor_idx_a = []
act_idx_a = []
ext_idx_a = []

for p, sp in enumerate(sa):
    if abs(sp - 2.0) < tol:
        cor_idx_a.append(p)
    elif abs(sp) < tol:
        ext_idx_a.append(p)
    else:
        act_idx_a.append(p)

ub, sb, vhb = linalg.svd(dm_bb)
numpy.linalg.norm(
    dm_bb - reduce(numpy.dot, [ub, numpy.diag(sb), vhb])
)

cor_idx_b = []
act_idx_b = []
ext_idx_b = []

for p, sp in enumerate(sb):
    if abs(sp - 2.0) < tol:
        cor_idx_b.append(p)
    elif abs(sp) < tol:
        ext_idx_b.append(p)
    else:
        act_idx_b.append(p)

In [6]:
print(cor_idx_a, act_idx_a, ext_idx_a)
print(sum(sa[cor_idx_a]))
print(sum(sa[act_idx_a]))
print(sum(sa[ext_idx_a]))

[0, 1] [2, 3, 4, 5] [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
3.9957846535639687
3.9999999999999956
0.0042153464360361005


In [7]:
print(cor_idx_b, act_idx_b, ext_idx_b)
print(sum(sa[cor_idx_b]))
print(sum(sa[act_idx_b]))
print(sum(sa[ext_idx_b]))

[0, 1] [2, 3, 4, 5] [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
3.9957846535639687
3.9999999999999956
0.0042153464360361005


In [8]:
c_ao_lo_a = c_ao_lo[:,idx_a]
c_pio_a   = reduce(numpy.dot, [c_ao_lo_a, ua])
npio = c_ao_lo_a.shape[1]

# for pp in range(npio):
#     tools.cubegen.orbital(mol, f'./cube/ch3_a_{pp}_{sa[pp]:4.2e}.cube', c_pio_a[:,pp])

c_ao_lo_b = c_ao_lo[:,idx_b]
c_pio_b   = reduce(numpy.dot, [c_ao_lo_b, ub])
npio = c_ao_lo_b.shape[1]

# for pp in range(npio):
#     tools.cubegen.orbital(mol, f'./cube/ch3_b_{pp}_{sb[pp]:4.2e}.cube', c_pio_b[:,pp])

In [9]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [c_pio_a.T, ovlp_ao, c_pio_a]) - numpy.eye(c_pio_a.shape[1])
)

In [10]:
nao, nmo = mf.mo_coeff.shape
coeff = numpy.empty([nao, nmo])

mo_occ = [0 for p in range(nmo)]

cor_list = []
cas_list = []
ext_list = []

count = 0

for p in cor_idx_a:
    pp = count
    mo_occ[pp] = 2 
    cor_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in cor_idx_b:
    pp = count
    mo_occ[pp] = 2
    cor_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1

for p in act_idx_a:
    pp = count
    cas_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in act_idx_b:
    pp = count
    cas_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1
    
for p in ext_idx_a:
    pp = count
    ext_list.append(pp)
    coeff[:, pp] = c_pio_a[:, p]
    count += 1

for p in ext_idx_b:
    pp = count
    ext_list.append(pp)
    coeff[:, pp] = c_pio_b[:, p]
    count += 1

assert count == nmo

In [11]:
assert numpy.linalg.norm(
    reduce(numpy.dot, [coeff.T, ovlp_ao, coeff]) - numpy.eye(nmo)
)


for pp in cas_list:
    print(f"Processing {pp}")
    tools.cubegen.orbital(mol, f'./cube/c2h6_{pp}.cube', coeff[:,pp])

Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Processing 10
Processing 11


In [12]:
cas_list

[4, 5, 6, 7, 8, 9, 10, 11]

In [13]:
from pyscf import mcscf

mf.mo_coeff = coeff
mycas = mcscf.CASCI(mf, 8, 8)
mo = mycas.sort_mo(cas_list)
mycas.kernel()
mycas.verbose = 4
mycas.analyze()

CASCI E = -78.099469906135  E(CI) = -16.282610405919  S^2 = 0.0000000
Natural occ [1.99127443 1.98898861 1.98517131 1.92827778 0.07192993 0.01381335
 0.01102154 0.00952305]
Natural orbital (expansion on meta-Lowdin AOs) in CAS space
               #1        #2        #3        #4        #5       
 0 C1 1s      -0.00000  -0.00000  -0.00123   0.00000   0.00000
 0 C1 2s       0.00000  -0.00000  -0.45377   0.00000  -0.00000
 0 C1 3s      -0.00000  -0.00000   0.04024  -0.00000   0.00000
 0 C1 2px      0.00000  -0.00000   0.00000   0.70441  -0.70391
 0 C1 2py     -0.48026  -0.56257   0.00000  -0.00000   0.00000
 0 C1 2pz      0.00000  -0.00000  -0.53419   0.00000  -0.00000
 0 C1 3px     -0.00000  -0.00000   0.00000   0.05057  -0.05379
 0 C1 3py      0.01604   0.04956  -0.00000   0.00000   0.00000
 0 C1 3pz      0.00000   0.00000   0.04708  -0.00000   0.00000
 0 C1 3dxy     0.00000   0.00000   0.00000   0.00000  -0.00000
 0 C1 3dyz     0.02541   0.01746   0.00000  -0.00000  -0.00000
 0 C1 3dz

(array([[ 1.00553449e+00, -5.80153250e-03, -1.74217652e-02, ...,
         -2.84216547e-10,  3.63338263e-04,  2.27226855e-04],
        [-5.80153250e-03,  1.42127919e-01,  1.12542591e-01, ...,
         -3.96974685e-09,  8.10475331e-04, -2.26447162e-03],
        [-1.74217652e-02,  1.12542591e-01,  1.06690715e-01, ...,
         -1.08301461e-09, -1.66295679e-03, -1.10536720e-03],
        ...,
        [-2.84216547e-10, -3.96974685e-09, -1.08301461e-09, ...,
          1.85462496e-04, -6.19395090e-11,  2.21375076e-11],
        [ 3.63338263e-04,  8.10475331e-04, -1.66295679e-03, ...,
         -6.19395090e-11,  1.10829217e-03, -4.33439891e-04],
        [ 2.27226855e-04, -2.26447162e-03, -1.10536720e-03, ...,
          2.21375076e-11, -4.33439891e-04,  4.11303635e-04]]),
 array([[ 1.00553455e+00, -5.80188233e-03, -1.74220434e-02, ...,
          1.26536939e-09,  3.63331707e-04,  2.27250217e-04],
        [-5.80188233e-03,  1.42125378e-01,  1.12542274e-01, ...,
          3.72268467e-09,  8.10437943e

In [14]:
ci = mycas.ci
if (ci is not None and
    (getattr(mycas.fcisolver, 'large_ci', None) or
     getattr(mycas.fcisolver, 'states_large_ci', None))):
    print('** Largest CI components **')
    print('  [alpha occ-orbitals] [beta occ-orbitals]            CI coefficient')
    res = mycas.fcisolver.large_ci(ci, mycas.ncas, mycas.nelecas,
                                    10e-2, return_strs=False)
    
    cc = 0.0
    
    for c,ia,ib in res:
        aa = [a+4 for a in ia]
        bb = [b+4 for b in ib]
        print('  %-20s %-30s %12.4e'%(aa, bb, c**2))
        cc += c**2
        
    print(f"cc = {cc:6.4f}")

** Largest CI components **
  [alpha occ-orbitals] [beta occ-orbitals]            CI coefficient
  [4, 5, 6, 8]         [4, 5, 6, 8]                     2.3282e-02
  [4, 5, 6, 8]         [4, 5, 8, 9]                     7.0960e-02
  [4, 5, 6, 8]         [4, 6, 8, 10]                    4.1336e-02
  [4, 5, 6, 8]         [4, 8, 9, 10]                    9.4047e-02
  [4, 5, 8, 9]         [4, 5, 6, 8]                     7.0941e-02
  [4, 5, 8, 9]         [4, 5, 8, 9]                     4.8774e-02
  [4, 5, 8, 9]         [4, 6, 8, 10]                    8.1435e-02
  [4, 5, 8, 9]         [4, 8, 9, 10]                    4.1335e-02
  [4, 6, 8, 10]        [4, 5, 6, 8]                     4.1335e-02
  [4, 6, 8, 10]        [4, 5, 8, 9]                     8.1457e-02
  [4, 6, 8, 10]        [4, 6, 8, 10]                    4.8774e-02
  [4, 6, 8, 10]        [4, 8, 9, 10]                    7.0961e-02
  [4, 8, 9, 10]        [4, 5, 6, 8]                     9.4020e-02
  [4, 8, 9, 10]        [4, 5, 8,